## Setup

In [46]:
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow import keras

from pathlib import Path
from IPython.display import display, Audio

import os
import librosa
import tensorflow as tf

In [36]:
DATASET_ROOT = os.path.join(os.path.expanduser("~"), "Downloads/16000_pcm_speeches")

# The folders in which we will put the audio samples and the noise samples
AUDIO_SUBFOLDER = "audio"
NOISE_SUBFOLDER = "noise"

DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)

In [37]:
# Percentage of samples to use for validation
VALID_SPLIT = 0.1

# Seed to use when shuffling the dataset and the noise
SHUFFLE_SEED = 43

# The sampling rate to use.
# This is the one used in all the audio samples.
# We will resample all the noise to this sampling rate.
# This will also be the output size of the audio wave samples
# (since all samples are of 1 second long)
SAMPLING_RATE = 16000

# The factor to multiply the noise with according to:
#   noisy_sample = sample + noise * prop * scale
#      where prop = sample_amplitude / noise_amplitude
SCALE = 0.5

BATCH_SIZE = 128
EPOCHS = 100

## Data preparation

In [38]:
# If folder `audio`, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)

# If folder `noise`, does not exist, create it, otherwise do nothing
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)

for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            # If folder is `audio` or `noise`, do nothing
            continue
        elif folder in ["other", "_background_noise_"]:
            # If folder is one of the folders that contains noise samples,
            # move it to the `noise` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            # Otherwise, it should be a speaker folder, then move it to
            # `audio` folder
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

## Noise preparation


In [41]:
# Get the list of all noise files
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print(
    "Found {} files belonging to {} directories".format(
        len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    ),'\n' , noise_paths
)

Found 6 files belonging to 2 directories 
 ['C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\other\\exercise_bike.wav', 'C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\other\\pink_noise.wav', 'C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\_background_noise_\\10convert.com_Audience-Claps_daSG5fwdA7o.wav', 'C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\_background_noise_\\doing_the_dishes.wav', 'C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\_background_noise_\\dude_miaowing.wav', 'C:\\Users\\FNafi\\Downloads\\16000_pcm_speeches\\noise\\_background_noise_\\running_tap.wav']


In [34]:
DATASET_NOISE_PATH 

'C:\\Users\\FNafi\\Downloads/16000_pcm_speeches\\noise'

In [55]:
import os
import subprocess
import shutil
import librosa

# Function to resample a WAV file to the desired sample rate
def resample_wav(file_path, desired_sample_rate):
    try:
        # Load the audio file and get its current sample rate
        audio_data, current_sample_rate = librosa.load(file_path, sr=None)

        # Check if the current sample rate is different from the desired rate
        if current_sample_rate != desired_sample_rate:
            # Resample the audio data to the desired sample rate
            audio_data_resampled = librosa.resample(audio_data, current_sample_rate, desired_sample_rate)

            # Save the resampled audio back to the original file path
            librosa.output.write_wav(file_path, audio_data_resampled, desired_sample_rate)
            print(f"Resampled: {file_path}")
        else:
            print(f"Already at the desired sample rate: {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Function to process all WAV files in a directory and its subdirectories
def process_wav_files(directory, desired_sample_rate):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(".wav"):
                file_path = os.path.join(root, file)

                # Execute ffprobe command to get the sample rate
                ffprobe_command = f"ffprobe -hide_banner -loglevel panic -show_streams {file_path}"
                result = subprocess.run(ffprobe_command, shell=True, stdout=subprocess.PIPE, text=True)
                sample_rate_info = [line for line in result.stdout.splitlines() if "sample_rate=" in line]
                
                if sample_rate_info:
                    sample_rate = int(sample_rate_info[0].split("sample_rate=")[1])
                    if sample_rate != desired_sample_rate:
                        # Use ffmpeg to resample the file to the desired sample rate
                        ffmpeg_command = f"ffmpeg -hide_banner -loglevel panic -y -i {file_path} -ar {desired_sample_rate} temp.wav"
                        subprocess.run(ffmpeg_command, shell=True)
                        shutil.move("temp.wav", file_path)
                        print(f"Resampled: {file_path}")
                    else:
                        print(f"Already at the desired sample rate: {file_path}")
                else:
                    print(f"Unable to retrieve sample rate for {file_path}")

# Main function
if __name__ == "__main__":
    NOISE_DIR = 'C:\\Users\\FNafi\\Downloads/16000_pcm_speeches\\noise'
    DESIRED_SAMPLE_RATE = 16000

    process_wav_files(NOISE_DIR, DESIRED_SAMPLE_RATE)


Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\other\exercise_bike.wav
Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\other\pink_noise.wav
Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\_background_noise_\10convert.com_Audience-Claps_daSG5fwdA7o.wav
Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\_background_noise_\doing_the_dishes.wav
Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\_background_noise_\dude_miaowing.wav
Unable to retrieve sample rate for C:\Users\FNafi\Downloads/16000_pcm_speeches\noise\_background_noise_\running_tap.wav


In [63]:
# Split noise into chunks of 16000 each
def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate >= SAMPLING_RATE:
        # Number of slices of 16000 each that can be generated from the noise sample
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

noises= noises[:354 ]
print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), noises.shape[0], noises.shape[1] // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


## Dataset generation


In [64]:
def paths_and_labels_to_dataset(audio_paths, labels):
    """Constructs a dataset of audios and labels."""
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        # Create a random tensor of the same size as audio ranging from
        # 0 to the number of noise stream samples that we have.
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)

        # Get the amplitude proportion between the audio and the noise
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)

        # Adding the rescaled noise to audio
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    # Since tf.signal.fft applies FFT on the innermost dimension,
    # we need to squeeze the dimensions and then expand them again
    # after FFT
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)

    # Return the absolute value of the first half of the FFT
    # which represents the positive frequencies
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])


# Get the list of audio file paths along with their corresponding labels

class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Processing speaker {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    speaker_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += speaker_sample_paths
    labels += [label] * len(speaker_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)

# Shuffle
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

# Split into training and validation
num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Using {} files for validation.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

# Create 2 datasets, one for training and the other for validation
train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)


# Add noise to the training set
train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Transform audio wave to the frequency domain using `audio_to_fft`
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Our class names: ['Benjamin_Netanyau', 'Jens_Stoltenberg', 'Julia_Gillard', 'Magaret_Tarcher', 'Nelson_Mandela']
Processing speaker Benjamin_Netanyau
Processing speaker Jens_Stoltenberg
Processing speaker Julia_Gillard
Processing speaker Magaret_Tarcher
Processing speaker Nelson_Mandela
Found 7501 files belonging to 5 classes.
Using 6751 files for training.
Using 750 files for validation.


In [65]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model.summary()

# Compile the model using Adam's default learning rate
model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Add callbacks:
# 'EarlyStopping' to stop training when the model is not enhancing anymore
# 'ModelCheckPoint' to always keep the model that has the best val_accuracy
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 8000, 1)]    0           []                               
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8000, 16)     64          ['input[0][0]']                  
                                                                                                  
 activation (Activation)        (None, 8000, 16)     0           ['conv1d_1[0][0]']               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 8000, 16)     784         ['activation[0][0]']             
                                                                                              

## Training

In [66]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/100
53/53 [==============================] - 89s 2s/step - loss: 1.2281 - accuracy: 0.6793 - val_loss: 0.2834 - val_accuracy: 0.8960
Epoch 2/100
53/53 [==============================] - 87s 2s/step - loss: 0.2832 - accuracy: 0.8893 - val_loss: 0.2749 - val_accuracy: 0.8827
Epoch 3/100
53/53 [==============================] - 87s 2s/step - loss: 0.2098 - accuracy: 0.9129 - val_loss: 0.1536 - val_accuracy: 0.9387
Epoch 4/100
53/53 [==============================] - 87s 2s/step - loss: 0.1482 - accuracy: 0.9430 - val_loss: 0.1397 - val_accuracy: 0.9373
Epoch 5/100
53/53 [==============================] - 88s 2s/step - loss: 0.1854 - accuracy: 0.9314 - val_loss: 0.1753 - val_accuracy: 0.9213
Epoch 6/100
53/53 [==============================] - 87s 2s/step - loss: 0.1200 - accuracy: 0.9530 - val_loss: 0.1052 - val_accuracy: 0.9493
Epoch 7/100
53/53 [==============================] - 87s 2s/step - loss: 0.1067 - accuracy: 0.9585 - val_loss: 0.0962 - val_accuracy: 0.9613
Epoch 8/100
5

## Evaluation

In [67]:
print(model.evaluate(valid_ds))


24/24 [==============================] - 3s 108ms/step - loss: 0.0578 - accuracy: 0.9800
[0.05782045051455498, 0.9800000190734863]


In [68]:
SAMPLES_TO_DISPLAY = 10

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

for audios, labels in test_ds.take(1):
    # Get the signal FFT
    ffts = audio_to_fft(audios)
    # Predict
    y_pred = model.predict(ffts)
    # Take random samples
    rnd = np.random.randint(0, BATCH_SIZE, SAMPLES_TO_DISPLAY)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(SAMPLES_TO_DISPLAY):
        # For every sample, print the true and predicted label
        # as well as run the voice with the noise
        print(
            "Speaker: {} - Predicted: {}".format(
                class_names[labels[index]],
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

4/4 [==============================] - 1s 104ms/step
Speaker: Benjamin_Netanyau - Predicted: Benjamin_Netanyau


Speaker: Julia_Gillard - Predicted: Julia_Gillard


Speaker: Jens_Stoltenberg - Predicted: Jens_Stoltenberg


Speaker: Jens_Stoltenberg - Predicted: Jens_Stoltenberg


Speaker: Magaret_Tarcher - Predicted: Magaret_Tarcher


Speaker: Nelson_Mandela - Predicted: Nelson_Mandela


Speaker: Benjamin_Netanyau - Predicted: Benjamin_Netanyau


Speaker: Magaret_Tarcher - Predicted: Magaret_Tarcher


Speaker: Nelson_Mandela - Predicted: Nelson_Mandela


Speaker: Julia_Gillard - Predicted: Julia_Gillard
